<a href="https://colab.research.google.com/github/parangatm/IoT-Security/blob/main/CA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re

In [ ]:
def get_control_blocks(code_lines):
    blocks = []
    current_block = []

    i=0
    while i < len(code_lines):

        ## the main function
        if "main() {" in code_lines[i]:
            ## "main()" and "{" both are in same line
            current_block.append(i)
            i = i+1
            continue
        elif "main()" in code_lines[i]:
            ## "main()" and "{" are in different line
            current_block.append(i)
            current_block.append(i+1)
            i = i+2
            continue

        ## if statement
        if "if" in code_lines[i]:
            blocks.append(current_block)
            ## new basic block for the "if" statement
            current_block = []
            occ = re.findall("if.*\(.*{", code_lines[i])
            # single line if (no curly brace = only one statement in the if block)
            if occ == []:
                current_block.append(i)
                current_block.append(i+1)
                blocks.append(current_block)
                current_block = []
                i=i+2
                continue
            # multi-line if (multiple statements inside if so {...})
            current_block.append(i)
            i = i+1
            continue

        ## else
        if "else" in code_lines[i]:
            blocks.append(current_block)
            ## new basic block for the "else" statement
            current_block = []
            occ = re.findall("else.*{", code_lines[i])
            # single line else (no curly brace = only one statement in the else block)
            if occ == []:
                current_block.append(i)
                current_block.append(i+1)
                blocks.append(current_block)
                current_block = []
                i=i+2
                continue
            # multi-line else (multiple statements inside else so {...})
            current_block.append(i)
            i = i+1
            continue

        ## ending brace - denotes ending of a control block such as if, else, for, switch
        if "}" in code_lines[i]:
            current_block.append(i)
            blocks.append(current_block)
            ## ending the ongoing basic block
            current_block = []
            i = i+1
            continue

        ## for
        if "for" in code_lines[i]:
            blocks.append(current_block)
            ## new basic block for the "for" statement
            current_block = []
            occ = re.findall("for.*\(.*{", code_lines[i])
            # single line for (no curly brace = only one statement in the for block)
            if occ == []:
                current_block.append(i)
                current_block.append(i+1)
                blocks.append(current_block)
                current_block = []
                i=i+2
                continue
            # multi-line for (multiple statements inside for so {...})
            current_block.append(i)
            i = i+1
            continue

        ## switch
        if "switch" in code_lines[i]:
            blocks.append(current_block)
            ## new basic block for the "switch" statement
            current_block = []
            current_block.append(i)
            i = i+1
            continue

        ## case
        if "case" in code_lines[i]:
            blocks.append(current_block)
            ## new basic block for the "case" statement
            current_block = []
            current_block.append(i)
            blocks.append(current_block)
            ## new basic block for the contents of the "case" statement
            current_block = []
            current_block.append(i+1)
            current_block.append(i+2)
            i = i+3
            continue

        ## default
        if "default" in code_lines[i]:
            blocks.append(current_block)
            ## new basic block for the "default" statement as well as its contents
            ## till the ending '}' brace
            current_block = []
            current_block.append(i)
            current_block.append(i+1)
            current_block.append(i+2)
            current_block.append(i+3)
            i = i+4
            continue

        #return
        if "return" in code_lines[i]:
            current_block.append(i)
            current_block.append(i+1)
            blocks.append(current_block)
            break

        #comments - to be skipped
        if "//" in code_lines[i]:
            i=i+1
            continue

        ## every non-control statement
        current_block.append(i)
        i = i+1

    return blocks

In [ ]:
## read the input file and return a list containing each line as a member
def get_code_lines(input_file):
    fc = open(input_file, 'r')
    code = fc.read()
    fc.close()
    code_lines = code.split('\n')
    return code_lines

In [ ]:
## print basic blocks in a readable-fashion
def print_basic_blocks(code_lines, blocks):

    ## Cleaning - removing some empty basic blocks - containing empty lines (whitespaces/newlines)
    for i,block in enumerate(blocks):
        if block == []:
            blocks.remove(block)
        else:
            for b in block:
                text = code_lines[b]
                if text == "":
                    blocks[i].remove(b)
                    #print("---empty popped---")

    #Cleaning
    for i,block in enumerate(blocks):
        if block == []:
            blocks.remove(block)
        else:
            for b in block:
                text = code_lines[b]
                if text == "":
                    blocks[i].remove(b)
                    #print("---empty popped---")

    ## merging nested endings (two sequential '}') into one block
    for i,block in enumerate(blocks):
        if len(block) == 1:
            stmt = code_lines[blocks[i][0]].strip()
            if stmt == "}":
                prev_block_end = code_lines[blocks[i-1][-1]].strip()
                if prev_block_end == "}":
                    blocks[i-1].append(block[0])
                    blocks.remove(block)

    for i, block in enumerate(blocks):
        for line in block:
            print("Block #",i,"\t",line+1,"\t",code_lines[line])
        print("-"*75)

In [ ]:
input_file_1 = 'myp.cpp'
code_lines_1 = get_code_lines(input_file_1)
basic_blocks_1 = get_control_blocks(code_lines_1)
print(basic_blocks_1)

[[0, 1, 2, 3], [4, 5], [], [6, 7], [8], [9, 10], [11, 12]]


In [ ]:
print_basic_blocks(code_lines_1, basic_blocks_1)

Block # 0 	 1 	 int main()
Block # 0 	 2 	 {
Block # 0 	 3 	 int a =0; 
Block # 0 	 4 	 int b = 2; 
---------------------------------------------------------------------------
Block # 1 	 5 	 if(a>-1)
Block # 1 	 6 	 	a = 3;
---------------------------------------------------------------------------
Block # 2 	 7 	 else 
Block # 2 	 8 	 	a = 4; 
---------------------------------------------------------------------------
Block # 3 	 9 	 a = b+ 5;
---------------------------------------------------------------------------
Block # 4 	 10 	 for (int i=0; i<4; i++)
Block # 4 	 11 	 	b = b+i; 
---------------------------------------------------------------------------
Block # 5 	 12 	 return 0;
Block # 5 	 13 	 }
---------------------------------------------------------------------------


In [ ]:
input_file_2 = 'advp.cpp'
code_lines_2 = get_code_lines(input_file_2)
basic_blocks_2 = get_control_blocks(code_lines_2)
print(basic_blocks_2)

[[0, 1, 2, 3], [4, 5], [6, 7, 8], [9, 10, 11], [12, 13], [14, 15], [16, 17, 18], [19], [20], [22, 23], [24, 25, 26], [27], [29], [30], [31, 32], [33], [34, 35], [36, 37, 38, 39, 40], [42, 43], [44, 45], [46, 47, 48], [49, 50, 51]]


In [ ]:
print_basic_blocks(code_lines_2, basic_blocks_2)

Block # 0 	 1 	 int main() {
Block # 0 	 2 	     int a = 0;
Block # 0 	 3 	     int b = 2;
---------------------------------------------------------------------------
Block # 1 	 5 	     if (a > -1) {
Block # 1 	 6 	         a = 3;
---------------------------------------------------------------------------
Block # 2 	 7 	     } else {
Block # 2 	 8 	         a = 4;
Block # 2 	 9 	     }
---------------------------------------------------------------------------
Block # 3 	 11 	     a = b + 5;
---------------------------------------------------------------------------
Block # 4 	 13 	     for (int i = 0; i < 4; i++) {
Block # 4 	 14 	         b = b + i;
---------------------------------------------------------------------------
Block # 5 	 15 	         if (b < 5) {
Block # 5 	 16 	             a = a * 2;
---------------------------------------------------------------------------
Block # 6 	 17 	         } else {
Block # 6 	 18 	             a = a / 2;
Block # 6 	 19 	         }
Block # 